<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [16]:
### import things
import pandas as pd
import copy
### read form the csv doc
data_train = pd.read_csv('train.csv')  
data_test = pd.read_csv('test.csv')
### trans to array
train_array = np.array(data_train)
test_array = np.array(data_test)
###############hyper parameter
use_normA=0
use_normy=0
###############hyper parameter

### process the A to be norm or not
if use_normA==1: 
    A_before_norm = train_array
    A=np.empty(np.shape(A_before_norm))
    for i in range(np.shape(A_before_norm)[1]):
        temp=A_before_norm[:,i]/np.linalg.norm(A_before_norm[:,i])
        A[:,i]=temp
else:
    A = train_array

### the A shape
[d,n] = A.shape


### aha! same as the lambda, the L-1 norm coefficent
lamb=1e-4

### the learning rate 0.15/(the proxi-L constant)
alpha=0.15/(0.25*max(sum(A*A)))

### loops
iters_outer=2
iters_inner=2*d

### count the number bingo
right=0

xx_0=np.zeros([85*67,67])
for i in range(67):
    xx_0[i*85:i*85+85][:,i]=1

for k in range(100):
    rand_index=np.random.randint(85*67)
#     print('rand_index',rand_index)
    rand_who=int(rand_index/85)
#     print('rand_who',rand_who)
    
    #### which y version you use, norm processed or the plain one 
    if use_normy==1:
        temp_y = test_array[:,rand_index].reshape(d,1)
        y=np.empty(np.shape(temp_y))
        y=temp_y/np.linalg.norm(temp_y)
    else:
        y=test_array[:,rand_index].reshape(d,1)
    
    ### initialize the three ws
    w_prev=np.array(np.zeros([n,1]))
    w_tilda=np.array(np.zeros([n,1]))
    ww_prev=np.array(np.zeros([n,1]))
    ### big loop
    for i in range(iters_outer):
        mu=(-A.T).dot(y-A.dot(w_tilda))/d
        ##small loop
        for j in range(iters_inner):
            ### choose a rand one from d 
            idx=np.random.randint(0,d)
            ### A transpose, need reshape
            A_idx=A[idx].T.reshape(n,1)
            ### g_prev and tilda
            g_prev=(-A_idx).dot(y[idx]-(A[idx].reshape(1,n)).dot(w_prev)) 
            g_tilda=(-A_idx).dot(y[idx]-(A[idx].reshape(1,n)).dot(w_tilda)) 
            ### update the parameter
            w_prev=w_prev-alpha*(g_prev-g_tilda+mu)
            ### the soft thresholding
            temp=w_prev-lamb*alpha
            temp=(temp+abs(temp))/2
            w_prev=np.sign(w_prev)*temp
            
            ### my try
#             w_prev[w_tilda*w_prev<0]=0
            ### sum the w_prev in the inner loop
            ww_prev+=w_prev
            
            

        ### assign the w_tilda for another loop    
        w_tilda=copy.deepcopy(ww_prev/iters_inner)
        w_prev=copy.deepcopy(w_tilda)


        
    res_min=1e9
    for i in range(np.shape(xx_0)[1]):
        temp=xx_0[:,i].reshape(5695,1)
        x_phi=w_tilda*temp
        res=sum((y-A.dot(x_phi))*(y-A.dot(x_phi)))
        if res<res_min:
            res_min=res
            index_min=i


    print('randwho and index_min',rand_who,index_min)
#     print('detect and rand_who',detect,rand_who)
#     print('rand_index',rand_index)
#     print('rand_who',rand_who)
#     print('*'*80)
#     if detect == rand_who:
#         right=right+1

    if rand_who == index_min:
        right=right+1


    
    
    

randwho and index_min 0 0
randwho and index_min 62 62
randwho and index_min 59 59
randwho and index_min 66 66
randwho and index_min 11 10
randwho and index_min 38 19
randwho and index_min 41 45
randwho and index_min 37 54
randwho and index_min 25 25
randwho and index_min 4 4
randwho and index_min 36 24
randwho and index_min 54 54
randwho and index_min 40 40
randwho and index_min 60 60
randwho and index_min 22 22
randwho and index_min 24 24
randwho and index_min 3 19
randwho and index_min 27 27
randwho and index_min 20 20
randwho and index_min 35 35
randwho and index_min 31 31
randwho and index_min 65 65
randwho and index_min 33 33
randwho and index_min 40 40
randwho and index_min 52 52
randwho and index_min 65 65
randwho and index_min 34 34
randwho and index_min 47 47
randwho and index_min 38 19
randwho and index_min 65 65
randwho and index_min 25 11
randwho and index_min 13 13
randwho and index_min 60 60
randwho and index_min 14 14
randwho and index_min 53 53
randwho and index_min 58 

In [15]:
right

75

In [11]:
# for i in range(np.shape(w_tilda)[0]):
#     print(w_tilda[i]*w_prev[i]<0)

w_tilda[w_tilda*w_prev<0]=0